**Importación de Librerias**

In [1]:
import tensorflow as tf
from tensorflow import keras
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from collections import Counter

import math
import numpy as np
import os
import time

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Descarga y preprocesado de datos**

In [3]:
#Cargar el texto del parlamentario1
text = open("/content/intervencionesCasado.txt", 'rb').read().decode(encoding='utf-8')
print('Longitud del texto:        {} carácteres'.format(len(text)))

vocab = sorted(set(text))

print ('El texto está compuesto de estos {} carácteres:'.format(len(vocab)))
print (vocab)

Longitud del texto:        105940 carácteres
El texto está compuesto de estos 92 carácteres:
['\n', ' ', '!', '%', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '«', '»', '¿', 'Á', 'á', 'é', 'í', 'ñ', 'ó', 'ö', 'ú', 'ü', '\u2009', '\u200c', '—', '―', '‘', '’', '…']


In [4]:
#Como las redes neuronales solo procesan valores numéricos, no letras. Traduciremos los caracteres a representación numérica.
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

for char,_ in zip(char2idx, range(len(vocab))):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))

  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '%' :   3,
  ',' :   4,
  '-' :   5,
  '.' :   6,
  '0' :   7,
  '1' :   8,
  '2' :   9,
  '3' :  10,
  '4' :  11,
  '5' :  12,
  '6' :  13,
  '7' :  14,
  '8' :  15,
  '9' :  16,
  ':' :  17,
  ';' :  18,
  '?' :  19,
  'A' :  20,
  'B' :  21,
  'C' :  22,
  'D' :  23,
  'E' :  24,
  'F' :  25,
  'G' :  26,
  'H' :  27,
  'I' :  28,
  'J' :  29,
  'L' :  30,
  'M' :  31,
  'N' :  32,
  'O' :  33,
  'P' :  34,
  'Q' :  35,
  'R' :  36,
  'S' :  37,
  'T' :  38,
  'U' :  39,
  'V' :  40,
  'X' :  41,
  'Y' :  42,
  'Z' :  43,
  '[' :  44,
  ']' :  45,
  'a' :  46,
  'b' :  47,
  'c' :  48,
  'd' :  49,
  'e' :  50,
  'f' :  51,
  'g' :  52,
  'h' :  53,
  'i' :  54,
  'j' :  55,
  'k' :  56,
  'l' :  57,
  'm' :  58,
  'n' :  59,
  'o' :  60,
  'p' :  61,
  'q' :  62,
  'r' :  63,
  's' :  64,
  't' :  65,
  'u' :  66,
  'v' :  67,
  'w' :  68,
  'x' :  69,
  'y' :  70,
  'z' :  71,
  '¡' :  72,
  '«' :  73,
  '»' :  74,
  '¿' :  75,
  'Á' :  76,

In [5]:
#Ahora pasaremos el texto a un array de enteros
text_as_int = np.array([char2idx[c] for c in text])

In [6]:
#Mostramos los 50 primeros caracteres del texto text_as_init
print ('texto : {}'.format(repr(text[:50])))
print ('{}'.format(repr(text_as_int[:50])))

texto : 'Señor Sánchez, sus recetas económicas son tan creí'
array([37, 50, 80, 60, 63,  1, 37, 77, 59, 48, 53, 50, 71,  4,  1, 64, 66,
       64,  1, 63, 50, 48, 50, 65, 46, 64,  1, 50, 48, 60, 59, 81, 58, 54,
       48, 46, 64,  1, 64, 60, 59,  1, 65, 46, 59,  1, 48, 63, 50, 79])


**Preparación de los datos para entrenar la RNN**

In [7]:
#Para entrenar el modelo creamos un conjunto de datos con el contenido de text_as_init. Para ello utilizamos la función tf.data.Dataset.from_tensor_slices.
#A este conjunto de datos lo dividiremos en secuencias de seq_length+1 al aplicar el método batch()

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [8]:
#Comprobamos que las sequences son el texto dividido en 101 caracteres (mostramos 10 secuencias)
for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'Señor Sánchez, sus recetas económicas son tan creíbles como sus promesas electorales, y encima propon'
'en las mismas recetas fracasadas que nos llevaron a la peor crisis económica de nuestra historia: más'
' despilfarro, más déficit y más impuestos. Pero el Partido Popular es un partido de Estado y también '
'de Gobierno, aunque estemos temporalmente en la oposición. Por eso el lunes le ofrecí pactar los Pres'
'upuestos Generales si rompe con los independentistas, una oferta, por cierto, a la que usted no ha co'
'ntestado. Hoy tiene oportunidad de hacerlo, porque ya no tiene excusa. Usted eligió libremente a sus '
'socios para la investidura, pero si ahora elige otra vez a los mismos para los presupuestos es porque'
' quiere. ¿O cree que al señor Torra le importan los parados en España, o al señor Otegi los agriculto'
'res y ganaderos, o al señor Junqueras los pensionistas o los autónomos? Ya lo dijeron aquí, les impor'
'ta España un comino. Y parece que a usted también si acepta una

In [9]:
#Creamos una función que devolverá el conjunto de datos de entrenamiento (los datos de entrada como los datos de salida)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

#Aplicamos la función a todas las secuencias utilizando el método map()
dataset = sequences.map(split_input_target)

In [10]:
#Los dataset contienen un conjunto de parejas (100 caracteres del texto original, la correspondiente salida ). Vamos a mostrar la primera pareja.
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

  print(dataset)

Input data:  'Señor Sánchez, sus recetas económicas son tan creíbles como sus promesas electorales, y encima propo'
Target data: 'eñor Sánchez, sus recetas económicas son tan creíbles como sus promesas electorales, y encima propon'
<_MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>


In [11]:
#Agrupamos los dataset en batches de 64 . Así tendriamos los datos de entrenamiento con batches compuestos de 64 parejas de secuencias de 100 integers de 64 bits
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print (dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>


**Construcción del modelo RNN**

In [12]:
#Crearemos una función que cree un modelo RNN con tres capas
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = Sequential()
  #Añadimos la capa de tipo word embedding
  model.add(Embedding(input_dim=vocab_size,
                      output_dim=embedding_dim,
                      #batch_input_shape=[batch_size, None] Deprecated
                      ))
  #Añadimos la capa de tipo LSTM
  model.add(LSTM(rnn_units,
                 return_sequences=True,
                 stateful=True,
                 recurrent_initializer='glorot_uniform'))
  model.add(Dense(512, activation="relu"))
  model.add(Dropout(0.5))


  #Añadimos la capa de tipo Dense
  model.add(Dense(vocab_size))
  return model

In [13]:
#Creamos el modelo
embedding_dim = 256
rnn_units = 1024

model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:
#A continuación inspecionamos las dimenciones de los tensores
for input_example_batch, target_example_batch in dataset.take(1):
  print("Input:", input_example_batch.shape, "# (batch_size, sequence_length)")
  print("Target:", target_example_batch.shape, "# (batch_size, sequence_length)")

Input: (64, 100) # (batch_size, sequence_length)
Target: (64, 100) # (batch_size, sequence_length)


In [16]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print("Prediction : ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Prediction :  (64, 100, 92) # (batch_size, sequence_length, vocab_size)


In [17]:
#Obtenemos una muestra de la distribución de salida
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices_characters = tf.squeeze(sampled_indices,axis=-1).numpy()

In [18]:
print(sampled_indices_characters)

[55 22 67 20 44 67 25 43  4 78 52 83 70 19 43 78 10 16 42 33 11 61  2 78
 64 42 17 37 45  7 10 81 54 35 38  6 65 37  0 81 27 88 75 26 56 10  4 90
 60 23 42  0 43 32 71 20 75 23 66 35 38 14 36 46 74 18 49  7 67 59 10 53
 79 23 41 73 35  8 50 86  9 84 70 91 91 22 79 38 67 22 52 87 49 82 27 54
 76 23 41 47]


**Entrenamiento del modelo RNN**

In [19]:
#Creamos la función de perdida, usaremos el categorical pues estamos considerando datos categóricos
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [20]:
#Compilamos el modelo
model.compile(optimizer='adam', loss=loss)

In [21]:
#configuramos los checkpoints

checkpoint_dir = './training_checkpoints_Casado'

# nombre fichero
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [22]:
def calculate_bleu(reference_text, generated_text):

    """
    Calcula el BLEU score comparando el texto generado con el texto de referencia.
    """

    reference_tokens = [list(reference_text)]
    generated_tokens = list(generated_text)
    smooth_fn = SmoothingFunction().method1
    score = sentence_bleu(reference_tokens, generated_tokens, smoothing_function=smooth_fn)

    return score



def calculate_perplexity(predictions):
    """
    Calcula la perplejidad promedio de una secuencia basada en los logits de salida del modelo.
    """
    # Convertimos logits a probabilidades con softmax
    probs = tf.nn.softmax(predictions, axis=-1).numpy()

    # Obtenemos las probabilidades máximas (las más probables)
    max_probs = np.max(probs, axis=-1)

    # Calculamos el logaritmo negativo de las probabilidades
    log_probs = -np.log(max_probs + 1e-10)  # Evitamos log(0) con 1e-10

    # Calculamos la perplejidad: exponencial de la media de los logaritmos
    perplexity = np.exp(np.mean(log_probs))

    return perplexity



def compare_word_distribution(reference_text, generated_text):

    """
    Compara la distribución de palabras entre el texto generado y el de referencia.
    """

    ref_counter = Counter(reference_text.split())
    gen_counter = Counter(generated_text.split())
    total_ref = sum(ref_counter.values())
    total_gen = sum(gen_counter.values())

    ref_dist = {word: count / total_ref for word, count in ref_counter.items()}
    gen_dist = {word: count / total_gen for word, count in gen_counter.items()}



    kl_divergence = 0

    for word, prob in gen_dist.items():
        if word in ref_dist:
            kl_divergence += prob * math.log(prob / ref_dist[word])

    return kl_divergence


In [23]:
#Entrenamos el modelo
EPOCHS=225
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/225
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 100ms/step - loss: 3.8592
Epoch 2/225
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - loss: 3.0758
Epoch 3/225
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - loss: 2.7287
Epoch 4/225
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - loss: 2.5688
Epoch 5/225
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - loss: 2.4495
Epoch 6/225
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - loss: 2.3656
Epoch 7/225
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - loss: 2.2786
Epoch 8/225
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - loss: 2.1890
Epoch 9/225
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - loss: 2.1140
Epoch 10/225
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - loss: 2.0448
Epoch 11/225
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 102ms/step - loss: 1.9717
Epoch 12/225
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - loss: 1.9025
Epoch 13/225
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - loss: 1.8419
Epoch 14/225
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - loss: 1.7559
Epoch 15/225
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 107ms/step -

In [24]:
model.save("model_Casado.keras")

In [25]:
from keras.models import load_model
from keras import losses # Import the losses module

# Assuming your original loss function was, for example, binary_crossentropy
loaded_model = load_model("model_Casado.keras",
                          custom_objects={'loss': losses.sparse_categorical_crossentropy})
# or if it was a custom loss function
# loaded_model = load_model("model_paquita_100_2024.keras", custom_objects={'loss': my_custom_loss_function})

**Generación de texto del modelo RNN**

In [26]:
#Ya que tenemos entrenado los modelos vamos a usarlos para generar texto
#Vamos a reconstruir manualmente los modelos para modifical el batch y su peso para poner el último
model = build_model(len(vocab), embedding_dim, rnn_units, batch_size=1)
# Build the model before loading weights
# This creates the necessary layer variables to which weights can be assigned
# Provide an input shape that matches the expected input of your model
# for example:
# input_shape = (1, sequence_length) where sequence_length is the length of your input sequences
input_shape = (1, 100)  # Replace 100 with your actual sequence length
model.build(input_shape=input_shape) # Or model.build(tf.TensorShape([1, None]))

model.load_weights("model_Casado.keras")

# Genera logits para una entrada de prueba
sample_input = tf.expand_dims([char2idx[s] for s in "texto de prueba"], axis=0)
logits = model(sample_input)  # Salida sin aplicar softmax

# Calcula la perplejidad con la función corregida
perplexity_value = calculate_perplexity(logits)
print(f"Perplejidad: {perplexity_value}")

Perplejidad: 1.781034231185913


In [27]:
#Creamos una función generar_texto que generará texto a partir de una palabra de partida
def generate_text(model, start_string):

  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]

  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []


  temperature = 0.45

#  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)

      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()


      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [28]:
print(generate_text(model, start_string=u"Hoy en el pleno"))

Hoy en el pleno en la legislación en vigor. .
Gracias, señora presidenta. Antes se ha referido a algo parecido a la oposición, a la que trata a patadas. Usted cree que este es el escenario y el tablero de ajedrenti una nefasta política de compra del material sanitario. Somos también, según el FMI, la segunda econtes, a las fallecidos, alcanzaría la nueva moralidad. Mire a su alrededor, no les decira, disuelva el Parlamento y convoque elecciones. . Solo así podrá quizá indultarle la historia. Muchas gracias. .
Dice que no deja a nadie atrás, pero no estamos en la solidaridad interterritorial y de la igualdad entre españoles, y acaban con el currículo educativo con implementación a nivel nacional de un porcentaje mínimo, digo que lo debería costar el cargo a su fiscalía por habitante del mundo, 18 000; y somos el país con más sanitarios contagiados del mundo por culpa de la arrogancia y la incompetencia de Sánchez, pero también por sus personas de los españoles. Muchas gracias. .
Señor S

In [29]:
print(generate_text(model, start_string=u"Es inaceptable que este gobierno haya decidido"))

Es inaceptable que este gobierno haya decidido no intervenir. No interviene el presidente del Gobierno, pues no intervenimos nosotros. El primero, el Pacto Cajal por la sanidad, dentro de la comisión parlamentaria, que yo le propuse y, por tanto, la responsabilidad es suya. Lealtad no es patente de corso; unidad no es barra libre y no hay que reformarla, basta con mundo más españoles. .
Señor Sánchez, no nos haga comulga de la Constitución Gabriel Cisneros. ¿Va usted a sacar de la cárcel a doscientos terroristas para comprar videoja de todos los indicadores negativos a nivel internacional. Se lo digo sin parafrasearle, para que no se repitan cierres como los de Alcoa o Nissan y una comisión con autonomía de para que no se repitan cierres como los de Alcoa o Nissan y una comisión con autonomía de para destruir el diálogo por antonomasia que es nuestra Constitución. Ya dialogamos, ya nos reconciler los cuatro ha fallado. El primero era acabar con el paro, pero lo ha disparado a 5 millone

In [30]:
# Texto de referencia: toma un segmento del dataset original
reference_text = text[:1000]  # Ejemplo: primeros 1000 caracteres del texto original
generated_text = generate_text(model, start_string="Ejemplo de inicio")

# BLEU Score
bleu_score = calculate_bleu(reference_text, generated_text)
print(f"BLEU Score: {bleu_score}")

# Perplejidad
input_eval = [char2idx[s] for s in "Ejemplo de inicio"]
input_eval = tf.expand_dims(input_eval, 0)
predictions = model(input_eval)

perplexity = calculate_perplexity(predictions[0])
print(f"Perplejidad: {perplexity}")

# Comparación de Distribución de Palabras
kl_divergence = compare_word_distribution(reference_text, generated_text)
print(f"KL Divergence (Distribución de Palabras): {kl_divergence}")

BLEU Score: 0.5312040881919403
Perplejidad: 1.9012998342514038
KL Divergence (Distribución de Palabras): 0.0872636562408613
